<h2>Problem 1. Python / Generator functions</h2>

Следующая функция возвращает текущее и предыдущее значения в цикле:

In [1]:
def this_and_prev(iterable):
    iterator = iter(iterable)
    prev_item = None
    curr_item = next(iterator)
    for next_item in iterator:
        yield (prev_item, curr_item)
        prev_item = curr_item
        curr_item = next_item
    yield (prev_item, curr_item)

In [2]:
for i,j in this_and_prev( range(5) ): print i,j

None 0
0 1
1 2
2 3
3 4


По аналогии требуется написать функцию, которая будет возвращать текущее и следующее значения.

<i>Type your code below</i>

In [1]:
def this_and_next(iterable):
    iterator = iter(iterable)
    curr_item = next(iterator)
    for next_item in iterator:
        yield (curr_item, next_item)
        curr_item = next_item
        
    yield (curr_item, None)

In [2]:
for i,j in this_and_next( range(5) ): print i,j

0 1
1 2
2 3
3 4
4 None


<h2>Problem 2. SQL / Python</h2>

Есть следующая SQL таблица <b>sample_table</b>:
<table>
<tr><td>column name</td><td><b>driver_id</b></td> <td><b>start_timestamp</b></td> <td><b>status</b></td></tr>
<tr><td>data type</td><td><i>(String)</i></td><td><i>(String)</i></td><td><i>(String)</i></td></tr>
<tr><td>1</td><td>driver_id_1</td><td>2017-01-21 00:05</td><td>driving</td></tr>
<tr><td>2</td><td>driver_id_1</td><td>2017-01-21 00:09</td><td>waiting</td></tr>
<tr><td>...</td><td>...</td><td>...</td><td>...</td></tr>
<tr><td>k x n</td><td>driver_id_n</td><td>2017-01-21 23:49</td><td>transporting</td></tr>
</table>

* driver_id_i -- идентификатор i-го водителя
* start_timestamp -- время начала статуса, в котором находился водитель
* status -- статус, в котором находился водитель

Для простоты предположим, что по каждому водителю в таблице одинаковое число записей k.

<hr>
Табличка хранится в СУБД, которая умеет применять к данным функции, написанные на Python. Например, следующий код выполняет функцию ROW_NUMBER():

In [4]:
def row_number(driver_id, input_data):
    sorted_data = sorted(input_data, lambda x: x[0]) # сортируем список входных данных по дате
    result = []
    row_number = 0
    while row_number <= range( 0, len(input_data) ):
        row_data = {'row_number': row_number
                    , 'driver_id': driver_id
                    , 'start_timestamp': sorted_data[row_number][0]
                    , 'status': sorted_data[row_number][1]
                    }
        row_number += 1
        result.append(row_data)
    return result

In [ ]:
$row_number = Python::row_number(driver_id, input_data);

$raw = (
    SELECT 
            driver_id
            , start_timestamp
            , status
    FROM    sample_table
    );

$reduced = (
    REDUCE $raw
       ON  driver_id
    USING  $row_number((start_timestamp, status))
    );

SELECT * FROM $reduced;


<hr>
Результат выполненного запроса будет выглядеть как:
<table>
<tr><td>column name</td><td><b>row_number</b></td><td><b>driver_id</b></td> <td><b>start_timestamp</b></td> <td><b>status</b></td></tr>
<tr><td>data type</td><td><i>(Int32)</i></td><td><i>(String)</i></td><td><i>(String)</i></td><td><i>(String)</i></td></tr>
<tr><td>1</td><td>1</td><td>driver_id_1</td><td>2017-01-21 00:05</td><td>driving</td></tr>
<tr><td>2</td><td>2</td><td>driver_id_1</td><td>2017-01-21 00:09</td><td>waiting</td></tr>
<tr><td>...</td><td>...</td><td>...</td><td>...</td><td>...</td></tr>
<tr><td>k x n</td><td>k</td><td>driver_id_n</td><td>2017-01-21 23:49</td><td>transporting</td></tr>
</table>

<hr>
<b>Вопрос</b>: как нужно переписать код, чтобы реализовать функцию LEAD(), т.е. добавить запись следующего статуса водителя в соседней колонке? Для выполнения задания требуется переписать код.

<i>Type your code below</i>

In [ ]:
def this_and_next(iterable):
    iterator = iter(iterable)
    curr_item = next(iterator)
    for next_item in iterator:
        yield (curr_item, next_item)
        curr_item = next_item
        
    yield (curr_item, None)
    
def lead(driver_id, input_data):
    sorted_data = sorted(input_data, lambda x: x[0]) # сортируем список входных данных по дате
    result = []
    row_number = 0
    
### NEW ###
    status_generator = this_and_next((row[1] for row in sorted_data))
### END ###
    
    while row_number <= range( 0, len(input_data) ):
        curr_status, next_status = next(status_generator)
        row_data = {'row_number': row_number
                    , 'driver_id': driver_id
                    , 'start_timestamp': sorted_data[row_number][0]
### REMOVE          , 'status': sorted_data[row_number][1]
### NEW ###
                    , 'status': curr_status
                    , 'next_status': next_status
### END ###
                    }
        row_number += 1
        result.append(row_data)
    return result

<h2>Problem 4. SQL</h2>

Есть следующая таблица с заказами клиентов: 

<table>
<tr><td>column name</td><td><b>id</b></td> <td><b>client_id</b></td> <td><b>driver_id</b></td> <td><b>timestamp</b></td> <td><b>cost</b></td> <td><b>payment_type</b></td> <td><b>status</b></td></tr>
<tr><td>data type</td><td><i>(String)</i></td> <td><i>(String)</i></td> <td><i>(String)</i></td> <td><i>(String)</i></td> <td><i>(Double)</i></td> <td><i>(String)</i></td> <td><i>(String)</i></td> </tr>
</table>

Нужно посчитать следующие метрики:
    1. Как процент выполнения заказов зависит от типа оплаты?
    2. Какой процент активных водителей совершает в неделю более 30 поездок?
    3. Какой процент клиентов, совершивших первую поездку за наличные впоследствии переходит на оплату картой?

<i>Type your answer below</i>

1. Как процент выполнения заказов зависит от типа оплаты?

_Предполагаем, что столбец ID соответствует поездке (выполненной или нет), что выполненному заказу соответствует значение  STATUS = 'transporting' и оно единственное для каждой поездки._

In [ ]:
SELECT sample_table.payment_type, 
    AVG(CASE 
            WHEN sample_table.status = 'transporting' THEN 1
            ELSE 0
        END) AS successful
FROM (SELECT id, MAX(timestamp) as last_ts FROM sample_table
    GROUP BY id) AS l
    INNER JOIN sample_table 
        ON l.id = sample_table.id, l.last_ts = sample_table.timestamp
GROUP BY payment_type;

2 Какой процент активных водителей совершает в неделю более 30 поездок?

_Считаем процент для каждой недели (пн-вс)_

In [ ]:
SELECT COUNT(
            SELECT driver_id 
            FROM weekly_trips
            WHERE num_trips > 30
            ) / COUNT(driver_id)
FROM (
    SELECT COUNT(status) AS num_trips, driver_id, YEARWEEK(timestamp, 7) AS week
    FROM sample_table
    WHERE status = 'transporting'
    GROUP BY YEARWEEK(timestamp, 7), driver_id
    HAVING COUNT( DISTINCT DAYOFWEEK(timestamp) ) = 7;
) AS weekly_trips
GROUP BY week

3 Какой процент клиентов, совершивших первую поездку за наличные впоследствии переходит на оплату картой?

_Берем первый и последний заказ клиента._

In [ ]:
SELECT AVG(
    CASE
        WHEN last_orders.payment_type = 'card' THEN 1
        ELSE 0
    END
    ) AS avg_switch_to_card

FROM (
    SELECT first_orders.client_id, sample_table.id
    FROM ( 
        SELECT client_id, MIN(timestamp) AS timestamp, status
        FROM sample_table 
        WHERE status = 'transporting'
        GROUP BY client_id 
    ) AS first_orders
    INNER JOIN sample_table 
    ON first_orders.client_id = sample_table.client_id
        AND first_orders.status = sample_table.status
        AND first_orders.timestamp = sample_table.timestamp 
    WHERE sample_table.payment_type = 'cash'
) AS first_cash_orders

INNER JOIN (
    SELECT last_orders.client_id, sample_table.payment_type, sample_table.id
    FROM (
        SELECT client_id, MAX(timestamp) AS timestamp, status
        FROM sample_table 
        WHERE status = 'transporting'
        GROUP BY client_id 
    ) AS last_orders
    INNER JOIN sample_table 
    ON last_orders.client_id = sample_table.client_id
        AND last_orders.status = sample_table.status
        AND last_orders.timestamp = sample_table.timestamp
) AS last_orders
ON first_cash_orders.client_id = last_orders.client_id
WHERE first_cash_orders.id <> last_orders.id

<h2>Problem 5. Algorithms</h2>

Город порезан на "квадраты". В момент t возникает точка на карте в пределах выделенной зоны (большого квадрата). Определить в какой из малых квадратов она попадет можно, например, с помощью перебора (bruteforce), который в среднем будет решать задачу за линейное время. Какой более эффективный алгоритм можно предложить для решения данной задачи и за какое время он будет ее решать?

Для выполнения задания не требуется писать код, можно описать логику алгоритма в 5-10 предложениях.

<img src="https://image.ibb.co/k3kzOk/2017_02_14_18_08_33.png" style="width:50%;height:50%;">

<i>Type your answer below</i>

Можно определить квадрат как пересечение вертикальной и горизонтальной полосы. Каждой вертикальной/горизонтальной полосе присвоим число соответствующее левой долготе / верхней широте. Тогда сначала мы проверим,  какой горизонтальной/вертикальной полосе соответствует широта точки, затем найдем нужный квадрат в таблице соответствия.
Для определения нужной полосы воспользуемся бинарным поиском: встаем в середину интервала и сравнением координаты точки с координатой полосы отсекаем половину интервала, пока не останется единственный кандидат. 
Полоса определяется за O(logN) время, поэтому две полосы - также O(logN), если соответствующий квадрат в таблице находится быстро, то и весь алгоритм имеет скорость O(log N), где N - число квадратов

<h2>Problem 6. A/B Testing</h2>

Необходимо понять, как прохождение обучения работе с приложением влияет на конверсию водителей из заявки на сайте (лид) в первую поездку (начало работы). Среди 1200 лидов прошедших обучение первую поездку сделали 370, среди группы не прошедшей обучение поехали 1250 из 4500 водителей. Какое решение вы бы приняли и почему?

Допустим, эксперимент показал, что конверсия выросла. Рассматривается возможность сделать обучение обязательным. Как это повлияет на показатели привлечения? Можно ли принимать это решение основываясь только на конверсии?

Следующий шаг эксперимента - в дополнение к конверсии нужно сравнить выручку, которую приносит водитель за первый месяц работы. Как правильно рассчитать эту выручку? Допустим, в группе с обучением, средняя выручка составила 52к рублей, а в группе без обучения 49к. Как бы вы принимали это решение основываясь на выручке и конверсии? Какой KPI на ваш взгляд важнее и почему? Меняется ли что-то в статистическом подходе к сравнению при переходе от конверсии к выручке?

<i>Type your answer below</i>

_Какое решение вы бы приняли и почему?_

Решение: протестируем гипотезу о равенстве конверсий. Также предположим, что вероятность конверсии одинакова для каждого водителя внутри одной группы. Альтернативная гипотеза:  конверсии разные.

In [5]:
import scipy.stats as st
m1 = 370/1200.
m2 = 1250/4500.
# H0: m1 = m2 (v1 = v2)
m = (370+1250)/(1200+4500.)
v = m * (1-m)
t = (m1 - m2) / (v * (1/1200. + 1/4500.))**0.5
df = 1200 + 4500 - 2
p_val = (1-st.t.cdf(t, df = df))/2
print "H0 отвергается при уровне значимости выше:", round(p_val, 4)
print "разница конверсий:", m1 - m2

H0 отвергается при уровне значимости выше: 0.0093
разница конверсий: 0.0305555555556


Разница статистически значима, но водители в двух выборках могут изначально находиться в разных распределениях, например, если прохождение обучения было добровольным: тогда обучение могли пройти водители, более склонные к тому, чтобы совершить первую поездку. 

В таком случае информации для принятия решения недостаточно. Как минимум, надо знать по какому алгоритму водители распределились на группы. Тогда есть несколько вариантов для тестирования гипотезы: 
1. Вариант с отсутствием доступа к обучению для одной из групп водителей. Такой тест вряд ли будет применен на практике.
2. Для одной группы обучение обязательно, для другой - по желанию. В данном случае в первой группе может наблюдаться отток из-за нежелания проходить обучение, но более высокая конверсия. Если дизайн эксперимента был таким, то цифры, приведенные выше, не говорят нам об оттоке. 

_Допустим, эксперимент показал, что конверсия выросла. Рассматривается возможность сделать обучение обязательным. Как это повлияет на показатели привлечения? Можно ли принимать это решение основываясь только на конверсии?_

Если заявка не считается заполненной без прохождения обучения работе с приложением или обязательность прохождения обучения становится известной на этапе заполнения заявки, часть водителей может отсеяться еще на этапе заполнения заявки, поэтому помимо конверсии нужно знать, как посещения страницы с заявкой конвертируются в заполненные заявки.

_Следующий шаг эксперимента - в дополнение к конверсии нужно сравнить выручку, которую приносит водитель за первый месяц работы. Как правильно рассчитать эту выручку? _

Возможно, непрохождение обучения увеличивает время между подачей заявки и первой поездкой, если водитель пытается разобраться с приложением самостоятельно. Если мы хотим исключить этот эффект, то можем посчитать выручку за 30 дней начиная с первой поездки. 

_Какой KPI на ваш взгляд важнее и почему?_

Если разница в выручке действительно отражает эффект обучения, то относительная важность KPI обусловлена как минимум двумя факторами:
1. Является ли более низкая выручка водителей в первый месяц существенной причиной оттока.
2. Сохраняется ли разница в выручке и далее. 

Если 1. 2. верно, то более низкая доля заполненных заявок при обязательном обучении компенсируется более высоким удержанием новых водителей. 
* Также, если водители, ушедшие после первого месяца работы, возращаются с меньшей вероятностью, чем водители, не заполнившие заявку, то возможно, выручка в первый месяц является более важным индикатором. С другой стороны, разница в выручке между двумя группами должна исчезнуть, как только водители полностью освоят приложение.
* Помимо этого, есть вероятность, что водители, не прошедшие обучение - это более занятые люди, которые просто отложили обучение на потом. Если они более занятые (например, параллельно работают в другом месте), они также склонны реже работать водителями, и тогда по сути разница в 3к означает лишь априорную разницу между двумя группами водителей, но не неумение работать с приложением.
* В итоге сложно сказать, какой KPI важнее, так как оба показателя являются прямым или косвенным индикатором оттока. Конверсия более понятна, т.к. для понимания эффекта изменения выручки на бизнес нужно проводить дополнительные расчеты. Вместе с тем фактом, что разница в выручке может нивелироваться, я бы отдал предпочтение конверсии.

_ Меняется ли что-то в статистическом подходе к сравнению при переходе от конверсии к выручке?_

При переходе от конверсии к выручке нам нужно также знать дисперсию выручки, поскольку ее нельзя выразить через среднее значение как в случае с конверсией.


<h2>Problem 7. Efficiency</h2>

В часы пик количество желающих воспользоваться такси резко возрастает и машин начинает не хватать. Для того чтобы обеспечить надежность сервиса в платформу заложен механизм балансировки спроса и предложения через динамическое ценообразование (surge pricing). 
1. От чего должен зависеть повышающий коэффициент (surge) и почему? Предложите алгоритм управления surge коэффициентом. 
2. Какие граничные условия вы бы предложили в качестве целевых (если коэффициент слишком низкий многие люди не могут уехать; если коэффициент слишком высокий - никто не хочет ехать)? 
3. Какие метрики нужно отслеживать чтобы понять, что алгоритм А работает лучше чем алгоритм Б?

<i>Type your answer below</i>

Коэффициент зависит от:
* Ожидаемого числа пользователей, открывших приложение: это число прямо пропорционально числу заказов в данном районе в данный период времени, на которые мы хотим повлиять. В теории это число можно оценить на исторических данных. В свою очередь, ожидаемое число пользователей зависит от:
    * времени и места: с утра повышенный спрос может возникнуть в спальных районах, а вечером - в районах крупных офисных центров, по выходным - в местах проведение развлекательных мероприятий.
* Ожидаемого числа свободных водителей в районе. Возможно, на относительно коротком промежутке времени это число не будет меняться, но есть периоды, когда число водителей резко растет или падает (например, по утрам или вечерам), это число зависит от:
    * также от времени и места: 
        * водители предпочитают работать днем, а не ночью;
        * в случае большой разницы между числом заказов в данный район и из данного района большинство водителей будет тратить дополнительное время на то, чтобы добраться до данного района. Если дополнительные затраты на дорогу не окупают повышенную вероятность заказа, коэффициент создаст дополнительный стимул.
* От цен конкурентов: если коэффициент слишком высокий, это поможет сбалансировать рынок в краткосрочном периоде, но может создать отток пользователей, т.к. многие из них пользуются сразу двумя или тремя приложениями, и в будущем будут отдавать предпочтение другим приложениям.


__Алгоритм__

Для упрощения будем считать, что коэффициент рассчитывается поинтервально для каджого сегмента определенного района/города
Также будем считать, что при расчете коэффициента в заданном сегменте учитывается только информация из определенного радиуса доступности (например, 4 км).
Логика следующего алгоритма основана на том, что мы можем оценить провести симуляцию рынка в заданном районе и оценить матожидание и крайние квантили ключевых метрик (мы хотим, чтобы коэффициент загруженности водителей был не ниже определенного уровня (и, например, чтобы лишь 1% водителей имел коэффициент загруженности ниже какого-то другого порога), та же логика для времени ожидания пользователей).

Поскольку мы не можем проводить симуляцию на бесконечно долгом периоде, мы можем оценить ее, например, на интервале ближайших 20 минут. Мы можем разбить этот интервал на маленькие отрезки (например 30 секунд) и переоценивать ее также каждые 30 секунд. Запишем динамику спроса и предложения:

_Предложение:_
Для упрощения будем считать, что число водителей в районе не меняется.
Чтобы получить время ожидания, мы можем записать следующую систему:
* $I_{t+1} = I_{t} + i_t - d_t$
* $D_{t+1} = D_{t} + d_t - t_t$
* $T_{t+1} = T_{t} + t_t - i_t$

Где I, D, T - число свободных, едущих к клиенту и везущих клиента водителей на опр. момент времени, (i, d, t) - число водителей, перешедших в соответствующее состояние на предыдущем интервале (30 секунд). Зная динамику этой системы, мы можем рассчитать, например, коэффициент загрузки водителей за следующие 20 минут.


Рассчитаем совокупное время в секундах: $\sum_{t=0}^{40} {t_t} * 30$. 
Чтобы посчитать среднюю загрузку, делим совокупное время на $(I_0 + D_0 + T_0)* 1200$ (предполагаем, что число водителей не меняется).

_Спрос:_
1. Допустим, что у нас есть оцененная на исторических данных функция распределения числа просмотров приложений по каждому сегменту района (допустим, сегмент - это квадрат 200 на 200м). Тогда мы можем сгенерировать случайное число из каждого сегмента. Если мы генерируем числа на интервалах в 30 секунд, сумма по всем сегментам в районе будет небольшой. Для каждого из сгенерированных пользователей также сгенерируем дальность поездки. 
2. Далее, рассчитаем вероятность того, что пользователь сделает заказ (при заданном коэффициенте, дальности поездки и т.п.) на оцененной функции спроса. 
3. Сгенерируем заказы (0 или 1) для каждого из пользователей, открывших приложение. Получим $o_t$ новых заказов на интервале t.

_Рынок:_
4. Распределим заказы по водителям (по готовому алгоритму или с помощью какой-то эвристики). Какие-то из водителей будут в этот момент свободными, а какие-то нет (что увеличит время ожидания клиента).
5. Рассчитаем время поездки водителя до клиента.
6. Рассчитаем время поездки клиента (на основе сгенерированной дальности поездки, пробок и т.п.)
7. Тогда, на момент времени $t+1$ статус водителей, получивших заказы, изменился ($d_{t} = o_t$) 
8. После этого, мы знаем, в какой момент времени $t+i$ статус водителя изменится с D на T и с T на I.
9. Также, мы рассчитали время ожидания для клиентов, сделавших заказ в момент $t$. 
10. Далее, мы повторяем все предыдущие шаги 40 раз (40 интервалов)
11. Далее, мы можем рассчитать все ключевые метрики за данное время: среднее время ожидания машины, загруженность водителей, долю пользователей, ожидающих больше К минут.
12. Можем провести эту симуляцию 100 раз и оценить вероятность, что какие-то из метрик превысят критические значения.
    * Если загруженность водителей слишком низкая, мы можем провести аналогичную симуляцию для пониженного коэффициента и подобрать оптимальный (например, методом золотого сечения). Если расчеты ресурсоемкие, можно опять же просто использовать эвристику (понизить коэффициент на 1%, если метрики неудовлетворительные). 
    * Если время ожидания для пользователя слишком велико, коэффициент можно повысить.
    
Оценим, как быстро система проводит подобный расчет:
* Допустим, что для каждого сегмента выбирается радиус в 4 км. Допустим, длина сегмента 200 метров. Тогда в радиусе любого сегмента находится $\pi *(4/0.2)^2 \approx 1250$ сегментов. 
* Для каждого сегмента мы проводим генерацию пользователей и заказов и получаем несколько новых заказов. Будем считать, что все остальные шаги занимают небольшую долю времени, и ими можно пренебречь.
* Тогда за одну итерацию мы сделаем порядка 1250 * 40 операций.
* За 100 итераций 1250 * 40 * 100 = 5 000 000 - операций по генерированию случайных чисел 
* Чтобы покрыть район размером 1000 сегментов, необходимо порядка  10^9 операций
Потенциально, расчет для сегмента может занимать доли секунды, так что теоретически есть шанс, что коэффициент успеет рассчитаться персонально для каждого пользователя после того, как он введет точку отправки. 



2.Если под граничными условиями понимаются значения метрик ожидания/загруженности, то мы можем посмотреть например на 95% квантиль $T_{95\%}$ времени ожидания машины и потребовать, чтобы в часы пик не менее 95% заказов имели время ожидания не более $T_{95\%} + K$ минут, где К - надбавка за то, что пользователи с пониманием отнесутся к дефициту водителей в часы пик (например К = 2 минуты). Возможно, это число также можно оценить исходя из истории оттока пользователй, систематически ожидаших такси слишком долго.
Для водителей ту же самую операцию можно провести для загруженности. Можно потребовать, чтобы 95% квантиль $L_{95\%}$ загрузки оставался на уровне не ниже среднего (без надбавок, т.к. повышенный коэффициент компенсирует простой водителям), 


3.Как уже упоминалось, отслеживать нужно среднее время ожидания заказа, загруженность водителей, долю заказов, в которых ожидание превысило некий разумный лимит (например, 15 минут). Дополнительно можно отслеживать волатильность повышающего коэффициента: пользователи могут жаловаться на скачкообразное изменение цен. Небольшая волатильность при тех же ключевых метриках это также хороший индикатор того, что алгоритм лучше предсказывает динамику рынка.